In [ ]:
%pip install numpy
%pip install pandas
%pip install matplotlib
%pip install seaborn
%pip install glob
%pip install pathlib

In [64]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from pathlib import Path

In [65]:
results_dataset = "./results/2025Assembly-DL.csv"
df = pd.read_csv(results_dataset)

print('='*50)
print(f"{df.loc[0, 'election_year']} {df.loc[0, 'election_state']} {df.loc[0, 'election_type']} Election Results")
print('='*50)

df['total_votes'] = df['evm_votes'] + df['postal_votes']
df.drop(['election_year', 'election_type'], axis=1, inplace=True)

display(df)

2025 DL Assembly Election Results


,election_state,constituency,serial_no,candidate,party,evm_votes,postal_votes,total_votes
0,DL,NERELA,1,ARUNA,Indian National Congress,6724,58,6782
1,DL,NERELA,2,RAJ KARAN KHATRI,Bharatiya Janata Party,86463,752,87215
2,DL,NERELA,3,SHARAD KUMAR,Aam Aadmi Party,78395,224,78619
3,DL,NERELA,4,ANIL KUMAR SINGH,Communist Party of India (Marxist-Leninist) (L...,328,0,328
4,DL,NERELA,5,MD KHALID,Aazad Samaj Party (Kanshi Ram),278,0,278
...,...,...,...,...,...,...,...,...
764,DL,KARAWAL NAGAR,12,H.M. HASHIM MALIK,Social Democratic Party Of India,491,0,491
765,DL,KARAWAL NAGAR,13,AJAY SINGH NEGI,Independent,1002,4,1006
766,DL,KARAWAL NAGAR,14,YOGESH SWAMY,Independent,228,1,229
767,DL,KARAWAL NAGAR,15,VISHAL SHARMA,Independent,173,0,173


In [66]:
print('='*50)
print("Political Affiliations including Independents")
print('='*50)
df['party'].value_counts()

Political Affiliations including Independents


party
Independent                             138
Indian National Congress                 70
Aam Aadmi Party                          70
None of the Above                        70
Bharatiya Janata Party                   68
                                       ... 
One Bharat Citizen Party                  1
Proutist Bloc, India                      1
Rashtravadi Loktantrik Party (India)      1
Bhartiya Krishak Dal                      1
Social Democratic Party Of India          1
Name: count, Length: 107, dtype: int64

In [67]:
top_n = 10

# Get top 2 candidates from each constituency
top_2_candidates = df.sort_values('total_votes', ascending=False).groupby('constituency').head(2)

# Create separate dataframes for winners and runners-up
winners = top_2_candidates.groupby('constituency').first().reset_index()
runners_up = top_2_candidates.groupby('constituency').last().reset_index()

# Combine winner and runner-up information
results = pd.DataFrame({
    'Constituency': winners['constituency'],
    'Candidate (Winner)': winners['candidate'],
    'Votes (Winner)': winners['total_votes'],
    'Party (Winner)': winners['party'],
    'Candidate (Runner-up)': runners_up['candidate'],
    'Votes (Runner-up)': runners_up['total_votes'],
    'Party (Runner-up)': runners_up['party'],
    'Margin': winners['total_votes'] - runners_up['total_votes']
})

print('='*80)
print(f"Top {top_n} Winning Candidates and Runners-up by Winner's Total Votes")
print('='*80)
results = results.sort_values('Votes (Winner)', ascending=False).reset_index(drop=True)

display(results.head(top_n))

Top 10 Winning Candidates and Runners-up by Winner's Total Votes


,Constituency,Candidate (Winner),Votes (Winner),Party (Winner),Candidate (Runner-up),Votes (Runner-up),Party (Runner-up),Margin
0,MATIALA,SANDEEP SEHRAWAT,146295,Bharatiya Janata Party,SUMESH SHOKEEN,117572,Aam Aadmi Party,28723
1,VIKASPURI,PANKAJ KUMAR SINGH,135564,Bharatiya Janata Party,MAHINDER YADAV,122688,Aam Aadmi Party,12876
2,BURARI,SANJEEV JHA,121181,Aam Aadmi Party,SHAILENDRA KUMAR,100580,Janata Dal (United),20601
3,BAWANA,RAVINDER INDRAJ SINGH,119515,Bharatiya Janata Party,JAI BHAGWAN UPKAR,88040,Aam Aadmi Party,31475
4,BADARPUR,RAM SINGH NETAJI,112991,Aam Aadmi Party,NARAYAN DUTT SHARMA,87103,Bharatiya Janata Party,25888
5,KARAWAL NAGAR,KAPIL MISHRA,107367,Bharatiya Janata Party,MANOJ KUMAR TYAGI,84012,Aam Aadmi Party,23355
6,KIRARI,ANIL JHA,105780,Aam Aadmi Party,BAJRANG SHUKLA,83909,Bharatiya Janata Party,21871
7,RITHALA,KULWANT RANA,104371,Bharatiya Janata Party,MOHINDER GOYAL,74755,Aam Aadmi Party,29616
8,UTTAM NAGAR,PAWAN SHARMA,103613,Bharatiya Janata Party,POSH BALYAN,73873,Aam Aadmi Party,29740
9,NAJAFGARH,NEELAM PAHALWAN,101708,Bharatiya Janata Party,TARUN KUMAR,72699,Aam Aadmi Party,29009


In [68]:
print('='*80)
print(f"Top {top_n} Winning Candidates and Runners-up by Winning Margin (smallest first)")
print('='*80)
results = results.sort_values('Margin', ascending=True).reset_index(drop=True)

display(results.head(top_n))

Top 10 Winning Candidates and Runners-up by Winning Margin (smallest first)


,Constituency,Candidate (Winner),Votes (Winner),Party (Winner),Candidate (Runner-up),Votes (Runner-up),Party (Runner-up),Margin
0,SANGAM VIHAR,CHANDAN KUMAR CHOUDHARY,54049,Bharatiya Janata Party,DINESH MOHANIYA,53705,Aam Aadmi Party,344
1,TRILOKPURI,RAVI KANT,58217,Bharatiya Janata Party,ANJANA PARCHA,57825,Aam Aadmi Party,392
2,JANGPURA,TARVINDER SINGH MARWAH,38859,Bharatiya Janata Party,MANISH SISODIA,38184,Aam Aadmi Party,675
3,TIMARPUR,SURYA PRAKASH KHATRI,55941,Bharatiya Janata Party,SURINDER PAL SINGH (BITTOO),54773,Aam Aadmi Party,1168
4,RAJINDER NAGAR,UMANG BAJAJ,46671,Bharatiya Janata Party,DURGESH PATHAK,45440,Aam Aadmi Party,1231
5,MEHRAULI,GAJENDER SINGH YADAV,48349,Bharatiya Janata Party,MAHENDER CHAUDHARY,46567,Aam Aadmi Party,1782
6,DELHI CANTT,VIRENDER SINGH KADIAN,22191,Aam Aadmi Party,BHUVAN TANWAR,20162,Bharatiya Janata Party,2029
7,MALVIYA NAGAR,SATISH UPADHYAY,39564,Bharatiya Janata Party,SOMNATH BHARTI,37433,Aam Aadmi Party,2131
8,GREATER KAILASH,SHIKHA ROY,49594,Bharatiya Janata Party,SAURABH BHARADWAJ,46406,Aam Aadmi Party,3188
9,KALKAJI,ATISHI,52154,Aam Aadmi Party,RAMESH BIDHURI,48633,Bharatiya Janata Party,3521


In [69]:
# Display party-wise vote shares up to NOTA or top_n parties
top_n = 10
total_votes = df['total_votes'].sum()
vote_shares = df.groupby('party')['total_votes'].sum().sort_values(ascending=False).reset_index()
vote_shares.rename(columns={'total_votes': 'Total Votes Won'}, inplace=True)

# Add vote share percentage
vote_shares['Vote Share %'] = (vote_shares['Total Votes Won'] / total_votes * 100).round(2)

# Determine display cutoff (either NOTA position or top_n)
nota_position = vote_shares[vote_shares['party'] == 'None of the Above'].index[0]
display_upto = min(nota_position+1, top_n)

print('='*25)
print(f"Party-wise Vote Share")
print('='*25)
display(vote_shares.head(display_upto))

Party-wise Vote Share


,party,Total Votes Won,Vote Share %
0,Bharatiya Janata Party,4323110,45.56
1,Aam Aadmi Party,4133898,43.57
2,Indian National Congress,601922,6.34
3,Janata Dal (United),100580,1.06
4,All India Majlis-E-Ittehadul Muslimeen,73032,0.77
5,Bahujan Samaj Party,55066,0.58
6,None of the Above,53738,0.57


In [70]:
df[['constituency', 'candidate', 'party', 'total_votes']].sort_values(by='total_votes', ascending=False).groupby('constituency').agg(list)

,candidate,party,total_votes
constituency,,,
ADARSH NAGAR,"[RAJ KUMAR BHATIA, MUKESH KUMAR GOEL, SHIVANK ...","[Bharatiya Janata Party, Aam Aadmi Party, Indi...","[52510, 41028, 5460, 377, 344, 140, 133, 116, ..."
AMBEDKAR NAGAR,"[DR. AJAY DUTT, KHUSHI RAM CHUNAR, JAI PRAKASH...","[Aam Aadmi Party, Bharatiya Janata Party, Indi...","[46285, 42055, 7172, 498, 397, 302, 261, 102, ..."
BABARPUR,"[GOPAL RAI, ANIL KUMAR VASHISHT, MOHD ISHRAQ K...","[Aam Aadmi Party, Bharatiya Janata Party, Indi...","[76192, 57198, 8797, 474, 201, 124, 93, 89, 76]"
BADARPUR,"[RAM SINGH NETAJI, NARAYAN DUTT SHARMA, ARJUN ...","[Aam Aadmi Party, Bharatiya Janata Party, Indi...","[112991, 87103, 3145, 1965, 915, 466, 367, 322..."
BADLI,"[AAHIR DEEPAK CHAUDHARYY, AJESH YADAV, DEVENDE...","[Bharatiya Janata Party, Aam Aadmi Party, Indi...","[61192, 46029, 41071, 725, 692, 250, 235, 214,..."
...,...,...,...
TUGHLAKABAD,"[SAHI RAM, ROHTASH KUMAR, VIRENDER SINGH, AMJA...","[Aam Aadmi Party, Bharatiya Janata Party, Indi...","[62155, 47444, 2313, 1606, 536, 323, 199, 143,..."
UTTAM NAGAR,"[PAWAN SHARMA, POSH BALYAN, MUKESH SHARMA, NOT...","[Bharatiya Janata Party, Aam Aadmi Party, Indi...","[103613, 73873, 15565, 1217, 991, 206, 196, 18..."
VIKASPURI,"[PANKAJ KUMAR SINGH, MAHINDER YADAV, JITENDER ...","[Bharatiya Janata Party, Aam Aadmi Party, Indi...","[135564, 122688, 8465, 2642, 1460, 687, 443, 4..."


In [71]:
sc = pd.read_csv('states.csv')
sc.rename(columns={'state_name': 'election_state'}, inplace=True)

display(sc)

,state_code,election_state,state_status,population,region,districts,assembly_seats,parliamentary_seats
0,AN,Andaman and Nicobar Islands,UT,380581,South,3,0,1
1,AP,Andhra Pradesh,State,49577103,South,26,175,25
2,AR,Arunachal Pradesh,State,1383727,Northeast,25,60,2
3,AS,Assam,State,31205576,Northeast,35,126,14
4,BR,Bihar,State,104099452,East,38,243,40
5,CH,Chandigarh,UT,1055450,Northwest,1,0,1
6,CT,Chhattisgarh,State,25545198,Central,33,90,11
7,DN,Dadra and Nagar Haveli and Daman and Diu,UT,586956,West,3,0,2
8,DL,NCT of Delhi,UT,16787941,Northwest,11,70,7
9,GA,Goa,State,1540000,West,2,40,2


In [79]:
# Get all Assembly CSV files from results directory
results_dir = Path('./results')
csv_files = glob.glob(str(results_dir / '*Assembly*.csv'))

# Read and concatenate all CSV files
df_master = pd.concat([pd.read_csv(f) for f in sorted(csv_files)], ignore_index=True)
df_master['total_votes'] = df_master['evm_votes'] + df_master['postal_votes']

print('='*60)
print(f"Master DataFrame of Election Results on India Votes Data")
print('='*60)

display(df_master)

Master DataFrame of Election Results on India Votes Data


,election_year,election_type,election_state,constituency,serial_no,candidate,party,evm_votes,postal_votes,total_votes
0,2024,Assembly,HR,KALKA,1,OM PARKASH GUJJAR,Aam Aadmi Party,854,4,858
1,2024,Assembly,HR,KALKA,2,CHARAN SINGH,Bahujan Samaj Party,1368,6,1374
2,2024,Assembly,HR,KALKA,3,PARDEEP CHAUDHARY,Indian National Congress,49544,185,49729
3,2024,Assembly,HR,KALKA,4,SHAKTI RANI SHARMA,Bharatiya Janata Party,60497,115,60612
4,2024,Assembly,HR,KALKA,5,AMIT SHARMA,Independent,582,0,582
...,...,...,...,...,...,...,...,...,...,...
8564,2025,Assembly,DL,KARAWAL NAGAR,12,H.M. HASHIM MALIK,Social Democratic Party Of India,491,0,491
8565,2025,Assembly,DL,KARAWAL NAGAR,13,AJAY SINGH NEGI,Independent,1002,4,1006
8566,2025,Assembly,DL,KARAWAL NAGAR,14,YOGESH SWAMY,Independent,228,1,229
8567,2025,Assembly,DL,KARAWAL NAGAR,15,VISHAL SHARMA,Independent,173,0,173
